In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.vision import *
from sklearn.metrics import roc_auc_score
from torchvision.models import *

In [3]:
path = Path('../input/')
!ls {path}

densenet169  models		    subm_20.csv   test
dn269	     sample_submission.csv  _subm_30.csv  tmp
labels.csv   subm		    _subm_41.csv  train


In [4]:
#tfms = get_transforms(do_flip=True, flip_vert=True, max_zoom=.1,
#                      max_lighting=0.05, max_warp=0.)
tfms = get_transforms()
data = ImageDataBunch.from_csv(path, folder='train', ds_tfms=tfms, size=96, test='test', suffix='.tif', bs=16)
data.normalize(imagenet_stats)
#data.show_batch(rows=3, figsize=(6,6))

ImageDataBunch;

Train: LabelList
y: CategoryList (176020 items)
[Category 0, Category 1, Category 0, Category 0, Category 1]...
Path: ../input
x: ImageItemList (176020 items)
[Image (3, 96, 96), Image (3, 96, 96), Image (3, 96, 96), Image (3, 96, 96), Image (3, 96, 96)]...
Path: ../input;

Valid: LabelList
y: CategoryList (44005 items)
[Category 1, Category 0, Category 0, Category 0, Category 0]...
Path: ../input
x: ImageItemList (44005 items)
[Image (3, 96, 96), Image (3, 96, 96), Image (3, 96, 96), Image (3, 96, 96), Image (3, 96, 96)]...
Path: ../input;

Test: LabelList
y: EmptyLabelList (57458 items)
[EmptyLabel , EmptyLabel , EmptyLabel , EmptyLabel , EmptyLabel ]...
Path: .
x: ImageItemList (57458 items)
[Image (3, 96, 96), Image (3, 96, 96), Image (3, 96, 96), Image (3, 96, 96), Image (3, 96, 96)]...
Path: ../input

In [ ]:
def auc_score(y_pred,y_true,tens=True):
    score=roc_auc_score(y_true,torch.sigmoid(y_pred)[:,1])
    if tens:
        score=tensor(score)
    else:
        score=score
    return score

In [ ]:
learn = create_cnn(data, densenet169, metrics=[auc_score, accuracy])

/home/tom/anaconda3/lib/python3.7/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [ ]:
print(learn.summary())

Layer (type)         Output Shape         Param #    Trainable 
Conv2d               [16, 64, 48, 48]     9408       False     
______________________________________________________________________
BatchNorm2d          [16, 64, 48, 48]     128        True      
______________________________________________________________________
ReLU                 [16, 64, 48, 48]     0          False     
______________________________________________________________________
MaxPool2d            [16, 64, 24, 24]     0          False     
______________________________________________________________________
BatchNorm2d          [16, 64, 24, 24]     128        True      
______________________________________________________________________
ReLU                 [16, 64, 24, 24]     0          False     
______________________________________________________________________
Conv2d               [16, 128, 24, 24]    8192       False     
______________________________________________________________

In [ ]:
# lr_find(learn)
# learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(12, max_lr=.02)

epoch,train_loss,valid_loss,auc_score,accuracy


In [ ]:
learn.precompute=False
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(8, max_lr=slice(1e-4, 1e-2))

In [ ]:
learn.fit_one_cycle(8, max_lr=slice(1e-6, 1e-3))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(16)

In [ ]:
preds_t,_ = learn.TTA(ds_type=DatasetType.Test)

In [ ]:
def submit(name, sample_file, learn, preds_t):
    sample_ds = pd.read_csv(sample_file).set_index('id')
    clean_fname=np.vectorize(lambda fname: str(fname).split('/')[-1].split('.')[0])
    fname=clean_fname(learn.data.test_ds.items) #(data.test_ds.items) learn.data.test_ds.fnames
    fname=fname.astype(str)
    sample_ds.loc[fname,'label']=to_np(preds_t[:,1])
    sample_ds.to_csv(name)
    !kaggle competitions submit -c histopathologic-cancer-detection -f {name} -m "{name}"

In [ ]:
submit('HCD_1.csv', path/'sample_submission.csv', learn, preds_t)

In [ ]:
learn.export('dn169')

## The END